In [1]:
%pylab inline 
import pandas as pd
from neomodel import config
from neomodel import (config, StructuredNode, StringProperty, IntegerProperty,
    UniqueIdProperty, RelationshipTo)

Populating the interactive namespace from numpy and matplotlib


In [87]:
df = pd.read_pickle('2019-10-27_tweets_with_quotes.pkl')

In [88]:
def extract_text(s):
    if pd.isna(s):
        return None
    else:
        return s['translatedText']
df['translated_text'] = df['translated_text'].apply(extract_text)

def extract_geo(s, subtype):

    try:
        geo = s['derived']['locations'][0][subtype]
        return geo
    except KeyError:
        return None
    
def extract_latlon(s, latlon):
    if pd.isna(s):
        return None
    try:
        geo = s['coordinates']
        latlon = geo[latlon]
        return latlon
    except KeyError:
        return None

In [89]:
df_user = pd.DataFrame()
df_user['id'] = df['user'].apply(lambda s: s['id'])
df_user['name'] = df['user'].apply(lambda s: s['name'])
df_user['screen_name'] = df['user'].apply(lambda s: s['screen_name'])
df_user['followers_count'] = df['user'].apply(lambda s: s['followers_count'])
df_user['longlat'] =  df['user'].apply(lambda s: extract_geo(s, 'geo'))
df_user['locality'] =  df['user'].apply(lambda s: extract_geo(s, 'locality'))
df_user['region'] =  df['user'].apply(lambda s: extract_geo(s, 'region'))
df_user['region'] =  df['user'].apply(lambda s: extract_geo(s, 'sub_region'))
df_user['longitude'] =  df_user['longlat'].apply(lambda s: extract_latlon(s, 0))
df_user['latitude'] =  df_user['longlat'].apply(lambda s: extract_latlon(s, 1))


In [92]:
df_tweet = pd.DataFrame()

In [120]:
# df_user_mentions = pd.DataFrame()
df_new_user = pd.DataFrame()
for i in df['entities'].apply(lambda s: s['user_mentions'] if len(s['user_mentions']) > 0 else None).dropna():
    for e in i:
        if not any(df_user['id'].isin([e['id']])):
            df_new_user = df_new_user.append(
                pd.DataFrame(e)
            )

In [125]:
df_user = df_user.append(df_new_user.drop(['id_str', 'indices'], axis=1).drop_duplicates())

In [132]:
df.columns

Index(['contributors', 'coordinates', 'created_at', 'created_at_date',
       'display_text_range', 'entities', 'extended_entities', 'extended_tweet',
       'favorite_count', 'favorited', 'filter_level', 'full_text', 'geo', 'id',
       'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'matching_rules',
       'place', 'possibly_sensitive', 'quote_count', 'quoted_status',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status_permalink',
       'reply_count', 'retweet_count', 'retweeted', 'retweeted_status',
       'source', 'text', 'translated_text', 'truncated', 'user'],
      dtype='object')

In [158]:
df_user = df_user.drop_duplicates('id')

In [160]:
df['full_text'] = df['full_text'].fillna(df['text'])

In [166]:
df['entities'].apply(lambda s: s['hashtags'] if len(s['hashtags']) > 0 else None).dropna()

4           [{'indices': [93, 101], 'text': 'measles'}]
20      [{'indices': [38, 51], 'text': 'titamovesssz'}]
28    [{'indices': [59, 64], 'text': 'baby'}, {'indi...
30    [{'indices': [25, 35], 'text': 'iwassakit'}, {...
38           [{'indices': [77, 85], 'text': 'vaccine'}]
                            ...                        
0     [{'indices': [28, 39], 'text': 'HepatitisB'}, ...
0     [{'indices': [97, 112], 'text': 'OnMyWayToCheo...
0     [{'indices': [119, 137], 'text': 'ALDUB57thWee...
0     [{'indices': [58, 68], 'text': 'Dengvaxia'}, {...
0              [{'indices': [44, 50], 'text': 'Viber'}]
Name: entities, Length: 1335, dtype: object

In [172]:
df['user_id'] = df['user'].apply(lambda s: s['id'])

In [187]:
df.drop(['created_at'], axis=1).drop_duplicates('id').to_excel('tweets_2019-12-12.xlsx', index=False)

C:\Users\PC\.conda\envs\twitter-vaccine\lib\site-packages\xlsxwriter\worksheet.py:915: UserWarning: Ignoring URL 'https://t.co/ZTVKFGmKYA%20SHOW%20na%20naman%20si%20Sen.%20Dick%20Gordon.%20No.1%20pa%20naman%20ikaw%20sa%20senator%20list%20ko%20kaso%20sa%20napapanood%20ko%20ngayon%20sa%20dengvaxia%20hearing%20nag.eexplain%20pa%20ung%20mga%20ininvite%20na%20resource%20person%20eh%20binabara%20mo%20na%20agad%20kasi%20ayaw%20mo%20nung%20sinasabi%20mapa-Pro%20or%20anti%20admin%20man.' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\Users\PC\.conda\envs\twitter-vaccine\lib\site-packages\xlsxwriter\worksheet.py:915: UserWarning: Ignoring URL 'https://t.co/lcG5aoJZ8S
%22Acosta’s%20allegations%20have%20resulted in%20the%20decline%20in%20vaccine%20confidence%20&amp;%20a%20rise%20in%20cases%20of%20Measles%20&amp;%20other%20vaccine%20preventable%20diseases.
DOH%20noted%20that%20cases%20of%20measles%20&amp;%20rubella%20dramatically incre

In [186]:
df

,contributors,coordinates,created_at,created_at_date,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,...,reply_count,retweet_count,retweeted,retweeted_status,source,text,translated_text,truncated,user,user_id
0,None,None,2018-11-22 10:34:24+00:00,2018-11-22 10:34:24,NaN,"{'hashtags': [], 'symbols': [], 'urls': [], 'u...",NaN,NaN,2,False,...,0,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...","Hoy ka yawa kasakit sa gi vaccine sa ako, pero...",You are so sick of the vaccine that I am but y...,False,"{'contributors_enabled': False, 'created_at': ...",779948021236633600
1,None,"{'coordinates': [121.04515361, 14.39491723], '...",2017-08-22 10:58:20+00:00,2017-08-22 10:58:20,NaN,"{'hashtags': [], 'symbols': [], 'urls': [{'dis...",NaN,NaN,0,False,...,0,0,False,NaN,"<a href=""http://instagram.com"" rel=""nofollow"">...",Isla: Got my 1st dose of Flu vaccine earlier!!...,Island: Got my 1st dose of Flu vaccine earlier...,False,"{'contributors_enabled': False, 'created_at': ...",401792154
2,None,None,2016-02-06 03:58:00+00:00,2016-02-06 03:58:00,NaN,"{'hashtags': [], 'symbols': [], 'urls': [], 'u...",NaN,NaN,0,False,...,0,0,False,NaN,"<a href=""http://twitter.com/download/android"" ...",Dengue.,Dengue.,False,"{'contributors_enabled': False, 'created_at': ...",2380888027
3,None,None,2019-07-21 08:50:01+00:00,2019-07-21 08:50:01,"[0, 28]","{'hashtags': [], 'media': [{'display_url': 'pi...",{'media': [{'display_url': 'pic.twitter.com/EO...,NaN,2,False,...,1,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",Dengue Nurse feeeels hahaha! https://t.co/EOTt...,Dengue Nurse feeeels hahaha! https://t.co/EOTt...,False,"{'contributors_enabled': False, 'created_at': ...",1123939613645713413
4,None,None,2019-02-08 00:11:47+00:00,2019-02-08 00:11:47,"[0, 140]","{'hashtags': [{'indices': [93, 101], 'text': '...",NaN,"{'display_text_range': [0, 276], 'entities': {...",0,False,...,0,0,False,NaN,"<a href=""http://twitter.com/download/android"" ...",@WHOPhilippines Representative @GundoWeiler em...,@WHOPhilippines Representative @GundoWeiler em...,True,"{'contributors_enabled': False, 'created_at': ...",91276420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,None,Mon Nov 20 08:04:06 +0000 2017,2017-11-20 08:04:06,"[0, 64]","{'hashtags': [], 'media': [{'display_url': 'pi...",{'media': [{'display_url': 'pic.twitter.com/t4...,NaN,2103,False,...,21,838,False,NaN,"<a href=""https://buffer.com"" rel=""nofollow"">Bu...",10+ Daily Struggles Of Living With A Cat\nhttp...,None,False,"{'contributors_enabled': False, 'created_at': ...",16548023
0,None,None,Fri Feb 15 11:08:23 +0000 2019,2019-02-15 11:08:23,"[0, 120]","{'hashtags': [], 'media': [{'additional_media_...",{'media': [{'additional_media_info': {'monetiz...,NaN,18,False,...,0,12,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",WATCH: Duterte appeals to all Filipino parents...,None,False,"{'contributors_enabled': False, 'created_at': ...",27605336
0,None,None,Mon Aug 12 07:19:46 +0000 2019,2019-08-12 07:19:46,"[0, 140]","{'hashtags': [], 'symbols': [], 'urls': [{'dis...",NaN,"{'display_text_range': [0, 287], 'entities': {...",14,False,...,2,1,False,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Correct me if I am wrong. Two major reasons wh...,None,True,"{'contributors_enabled': False, 'created_at': ...",338899489
0,None,None,Sun Feb 10 11:36:17 +0000 2019,2019-02-10 11:36:17,NaN,"{'hashtags': [], 'symbols': [], 'urls': [{'dis...",NaN,"{'display_text_range': [0, 143], 'entities': {...",94,False,...,35,23,False,NaN,"<a href=""https://www.echobox.com"" rel=""nofollo...",A viral video shows employees of the Baguio Ci...,None,True,"{'contributors_enabled': False, 'created_at': ...",330826792
